In [1]:
import sys
sys.path.append('../')
import pytest
import xlwings as xw
import pandas as pd
from pylim import lim
pd.options.display.float_format = "{:,.2f}".format
from xlwings import Range, constants
from oilanalytics.freight import tce_calc, routes, route_consts

In [2]:
wb = xw.Book(r'C:\dev\oilanalytics\notebooks\Baltic Exchange Tanker TCE Calculator SEP.20.xlsx')
sh = wb.sheets['TCE Calculator']

In [3]:
route = 'tc14'

In [4]:
df = routes.lim_data(route, start_date="date is within 1 days")
df = df.iloc[[-2]]
df

,MGO,VLSFO,LSFO,HSFO,WorldScale,FlatRate
2021-10-06,692.00,552.00,515.00,464.75,75.00,17.96


In [5]:
ws = df.iloc[0].WorldScale
fr = df.iloc[0].FlatRate
mgo = df.iloc[0].MGO
vlsfo = df.iloc[0].VLSFO

In [6]:
routef = route.upper()
sh.range("b7").value = routef
sh.range("C7").value = ws
sh.range("D7").value = fr

In [7]:
if route in route_consts.route_using_sing_fuels:
    sh.range("C16").clear_contents()
    sh.range("C17").clear_contents()
    sh.range("C14").value = vlsfo
    # sh.range("C15").value = mgo
else:
    sh.range("C14").clear_contents()
    sh.range("C15").clear_contents()
    sh.range("C16").value = vlsfo
    sh.range("C17").value = mgo

In [8]:
loadportcost = sh.range("c60").value
dischargeportcost = sh.range("c61").value

In [9]:
dd = {
    'FlatRate': fr,
    'WorldScale': ws,
    'MGO': mgo-5,
    'VLSFO': vlsfo-5,    
    'HSFO': vlsfo-5, 
    'LoadPortCosts': loadportcost,
    'DischargePortCosts': dischargeportcost
}
dd

{'FlatRate': 17.96,
 'WorldScale': 75.0,
 'MGO': 687.0,
 'VLSFO': 547.0,
 'HSFO': 547.0,
 'LoadPortCosts': 37725.0,
 'DischargePortCosts': 66893.3317371004}

In [10]:
ds = df.index[0]
data = pd.DataFrame(dd, index=[pd.to_datetime(ds)])
res = tce_calc.calc(route, data)
res

,FlatRate,WorldScale,MGO,VLSFO,HSFO,LoadPortCosts,DischargePortCosts,Freight_USDMT,GrossFreight,BunkerCost,NetFreight,TotalExpenses,NetIncome,TCE
2021-10-06,17.96,75.00,687.00,547.00,547.00,"37,725.00","66,893.33",13.47,"511,860.00","570,553.00","499,063.50","675,171.33","-176,107.83","-4,370.47"


In [11]:
cmp = sh.range("B79:C86").value
cmp = pd.DataFrame(cmp).set_index(0)
cmp = cmp.rename(columns={1:'Baltic'})
cmp['Code'] = pd.Series()
cmp.loc['Gross Freight', 'Code'] = res['GrossFreight'].iloc[0]
cmp.loc['Bunker Cost', 'Code'] = res['BunkerCost'].iloc[0]
cmp.loc['Net Income', 'Code'] = res['NetIncome'].iloc[0]
cmp.loc['Total Expenses', 'Code'] = res['TotalExpenses'].iloc[0]
cmp.loc['Baltic TCE', 'Code'] = res['TCE'].iloc[0]
cmp

C:\Users\ui906879\AppData\Local\Temp\1/ipykernel_26264/2024915855.py:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  cmp['Code'] = pd.Series()


,Baltic,Code
0,,
Gross Freight,"511,860.00","511,860.00"
Net Freight,"499,063.50",NaN
Bunker Cost,"570,549.36","570,553.00"
Total Expenses,"675,167.69","675,171.33"
Net Income,"-176,104.19","-176,107.83"
- Gross,"-4,052.81",NaN
- Net,"-4,370.38",NaN
Baltic TCE,"-4,370.00","-4,370.47"


In [12]:
# gross freight
assert sh.range("C79").value == pytest.approx(res['GrossFreight'].iloc[0], abs=1e-1)

In [13]:
# bunker cost
assert sh.range("C81").value == pytest.approx(res['BunkerCost'].iloc[0], abs=1e-1)

AssertionError: 

In [ ]:
# tce
assert sh.range("C86").value == pytest.approx(res['TCE'].iloc[0], abs=1e-1)

In [14]:
print("assert res['GrossFreight'][ds] ==  pytest.approx(%.2f, abs=1e-1)" % res['GrossFreight'].iloc[0])
print("assert res['BunkerCost'][ds] ==  pytest.approx(%.2f, abs=1e-1)" % res['BunkerCost'].iloc[0])
print("assert res['TCE'][ds] ==  pytest.approx(%.2f, abs=1e-1)" % res['TCE'].iloc[0])

assert res['GrossFreight'][ds] ==  pytest.approx(511860.00, abs=1e-1)
assert res['BunkerCost'][ds] ==  pytest.approx(570553.00, abs=1e-1)
assert res['TCE'][ds] ==  pytest.approx(-4370.47, abs=1e-1)
